In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from nltk.stem import PorterStemmer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score
import os

training = pd.read_csv('classifier_data_10.csv', header = 0)
testing = pd.read_csv('deep_data.csv', header = 0, delimiter = ",")
testing.drop(['Unnamed: 2'], axis=1, inplace = True)
training.drop(['Unnamed: 3','Unnamed: 4'], axis=1, inplace = True)


training['description'] = training['description'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
# Removing punctuation
training['description'] = training['description'].str.replace('[^\w\s]','')
# Stop word removal
stop = stopwords.words('english')
training['description'] = training['description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Stemming
st = PorterStemmer()
training['description'] = training['description'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))



training['description'].head()

X=training[['description']]  # Features
y=training[['owner']]

X_test = testing[['description']]  # Features

words = set(nltk.corpus.words.words())

vectorizer = TfidfVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 200) 
                             
vectorizer.fit(X['description'])
vector = vectorizer.transform(X['description'])
train_data_features=vector.toarray()

forest = RandomForestClassifier(n_estimators = 150) 
forest = forest.fit( train_data_features, y['owner'])

vectorizer1 = TfidfVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 200) 
                             
vector1 = vectorizer1.fit_transform(X_test['description'].apply(lambda x: np.str_(x)))

test_data_features=vector1.toarray()

result = forest.predict(test_data_features)
output = pd.DataFrame( data={"id":testing["id"], "owner":result, "description":testing["description"]} )

output.to_csv('dataset3_TFIDF_result.csv', index=False, quoting=3 )